In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

train_files, train_targets = load_dataset('dogs/train')
valid_files, valid_targets = load_dataset('dogs/valid')
test_files, test_targets = load_dataset('dogs/test')

dog_names = [item[20:-1] for item in sorted(glob("dogs/train/*/"))]

# Let's check the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

In [9]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [3]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:05<00:00, 158.84it/s]


In [4]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Activation, Dense, Flatten
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Conv2D(16, (3, 3), padding='same', use_bias=False, input_shape=(224, 224, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(512, activation='relu'))
model.add(Dense(133, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 16)      432       
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 16)      48        
_________________________________________________________________
activation (Activation)      (None, 224, 224, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 56, 56, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 32)        4608      
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 32)        9

In [5]:
from keras.callbacks import ModelCheckpoint  

EPOCHS = 10
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=EPOCHS, batch_size=32, callbacks=[checkpointer], verbose=1)

Epoch 1/10
209/209 [==============================] - ETA: 0s - loss: 4.8497 - accuracy: 0.0184
Epoch 00001: val_loss improved from inf to 5.41804, saving model to saved_models/weights.best.from_scratch.hdf5
209/209 [==============================] - 53s 255ms/step - loss: 4.8497 - accuracy: 0.0184 - val_loss: 5.4180 - val_accuracy: 0.0108
Epoch 2/10
209/209 [==============================] - ETA: 0s - loss: 4.3895 - accuracy: 0.0458
Epoch 00002: val_loss improved from 5.41804 to 4.87286, saving model to saved_models/weights.best.from_scratch.hdf5
209/209 [==============================] - 53s 254ms/step - loss: 4.3895 - accuracy: 0.0458 - val_loss: 4.8729 - val_accuracy: 0.0180
Epoch 3/10
209/209 [==============================] - ETA: 0s - loss: 4.1245 - accuracy: 0.0680
Epoch 00003: val_loss did not improve from 4.87286
209/209 [==============================] - 53s 255ms/step - loss: 4.1245 - accuracy: 0.0680 - val_loss: 5.2800 - val_accuracy: 0.0251
Epoch 4/10
209/209 [===========

In [6]:
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]
# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 6.5789%


In [10]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50

def extract_VGG19(file_paths):
    tensors = paths_to_tensor(file_paths).astype('float32')
    preprocessed_input = preprocess_input_vgg19(tensors)
    return VGG19(weights='imagenet', include_top=False).predict(preprocessed_input, batch_size=32)

def extract_Resnet50(file_paths):
    tensors = paths_to_tensor(file_paths).astype('float32')
    preprocessed_input = preprocess_input_resnet50(tensors)
    return ResNet50(weights='imagenet', include_top=False).predict(preprocessed_input, batch_size=32)

In [8]:
train_vgg19 = extract_VGG19(train_files)
valid_vgg19 = extract_VGG19(valid_files)
test_vgg19 = extract_VGG19(test_files)
print("VGG19 shape", train_vgg19.shape[1:])

train_resnet50 = extract_Resnet50(train_files)
valid_resnet50 = extract_Resnet50(valid_files)
test_resnet50 = extract_Resnet50(test_files)
print("Resnet50 shape", train_resnet50.shape[1:])

  0%|          | 25/6680 [00:00<00:27, 244.98it/s]

VGG19 shape (7, 7, 512)


100%|██████████| 836/836 [00:05<00:00, 158.41it/s]


Resnet50 shape (7, 7, 2048)


In [9]:

from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers import Input, Dense
from keras.layers.core import Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.models import Model

def input_branch(input_shape=None):
    
    size = int(input_shape[2] / 4)
    
    branch_input = Input(shape=input_shape)
    branch = GlobalAveragePooling2D()(branch_input)
    branch = Dense(size, use_bias=False, kernel_initializer='uniform')(branch)
    branch = BatchNormalization()(branch)
    branch = Activation("relu")(branch)
    return branch, branch_input

vgg19_branch, vgg19_input = input_branch(input_shape=(7, 7, 512))
resnet50_branch, resnet50_input = input_branch(input_shape=(1, 1, 2048))
concatenate_branches = Concatenate()([vgg19_branch, resnet50_branch])
net = Dropout(0.3)(concatenate_branches)
net = Dense(640, use_bias=False, kernel_initializer='uniform')(net)
net = BatchNormalization()(net)
net = Activation("relu")(net)
net = Dropout(0.3)(net)
net = Dense(133, kernel_initializer='uniform', activation="softmax")(net)

model = Model(inputs=[vgg19_input, resnet50_input], outputs=[net])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 7, 7, 512)]  0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 1, 1, 2048)] 0                                            
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 512)          0           input_7[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 2048)         0           input_8[0][0]                    
_______________________________________________________________________________________

In [10]:
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='saved_models/bestmodel.hdf5', 
                               verbose=1, save_best_only=True)
model.fit([train_vgg19, train_resnet50], train_targets, 
          validation_data=([valid_vgg19, valid_resnet50], valid_targets),
          epochs=10, batch_size=4, callbacks=[checkpointer], verbose=1)


Epoch 1/10
1665/1670 [============================>.] - ETA: 0s - loss: 2.6659 - accuracy: 0.3592WARNING:tensorflow:Model was constructed with shape (None, 1, 1, 2048) for input Tensor("input_8:0", shape=(None, 1, 1, 2048), dtype=float32), but it was called on an input with incompatible shape (None, 7, 7, 2048).

Epoch 00001: val_loss improved from inf to 0.95433, saving model to saved_models/bestmodel.hdf5
1670/1670 [==============================] - 7s 4ms/step - loss: 2.6644 - accuracy: 0.3597 - val_loss: 0.9543 - val_accuracy: 0.7114
Epoch 2/10
1664/1670 [============================>.] - ETA: 0s - loss: 1.5450 - accuracy: 0.5666
Epoch 00002: val_loss improved from 0.95433 to 0.82596, saving model to saved_models/bestmodel.hdf5
1670/1670 [==============================] - 7s 4ms/step - loss: 1.5468 - accuracy: 0.5662 - val_loss: 0.8260 - val_accuracy: 0.7617
Epoch 3/10
1664/1670 [============================>.] - ETA: 0s - loss: 1.3343 - accuracy: 0.6206
Epoch 00003: val_loss impro

In [11]:
from sklearn.metrics import accuracy_score

predictions = model.predict([test_vgg19, test_resnet50])
breed_predictions = [np.argmax(prediction) for prediction in predictions]
breed_true_labels = [np.argmax(true_label) for true_label in test_targets]
print('Test accuracy: %.4f%%' % (accuracy_score(breed_true_labels, breed_predictions) * 100))

Test accuracy: 79.7847%


In [12]:
predic = model.predict([extract_VGG19(['dogs/test/027.Bloodhound/Bloodhound_01871.jpg']),extract_Resnet50(['dogs/test/027.Bloodhound/Bloodhound_01871.jpg'])])

100%|██████████| 1/1 [00:00<00:00, 469.74it/s]


In [13]:
print(np.argmax(predic))

26


In [11]:
model_json = model.to_json()
with open('CNN_dog_breed_model_v2.json','w') as json_file:
    json_file.write(model_json)

model.save_weights('CNN_dog_breed_model_v2.h5')
print('Saved model to disk')

NameError: name 'model' is not defined

In [ ]:
dog = {0:'아펜핀셔',1:'아프간 하운드',2:'에어데일 테리어',3:'아키타견',4:'알레스칸 말라뮤트',
      5:'아메리칸 에스키모',6:''}

In [15]:
# 종 분류 모델 가져오기
from tensorflow.compat.v2.keras.models import model_from_json

# json 파일 열기-v2
json_file = open('CNN_dog_breed_model_v2.json','r')
loaded_model_json = json_file.read()
json_file.close()

# json 파일로부터 model 로드하기
loaded_model = model_from_json(loaded_model_json)

# 로드한 model에 weight 로드하기
loaded_model.load_weights('CNN_dog_breed_model_v2.h5')

In [17]:
predic = loaded_model.predict([extract_VGG19(['cats/american_bulldog_29.jpg']),extract_Resnet50(['cats/american_bulldog_29.jpg'])])
print(np.argmax(predic))

100%|██████████| 1/1 [00:00<00:00, 196.30it/s]


100%|██████████| 1/1 [00:00<00:00, 193.72it/s]


49
